In [1]:
import mysqlconnect

In [2]:
import iql

In [3]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [4]:
start = '2017-07-01'
end = '2017-07-30'
iql_end = '2017-08-01'

In [5]:
def get_average_position(start, end):
    qry = ("SELECT country, SUM(impressions*average_position)/SUM(impressions) as average_position FROM "
           "t_google_search_average_position WHERE country in ('US', 'UK', 'CA', 'DE', 'AU', 'FR', 'NL', "
           "'JP', 'IN', 'BR') AND activity_date between '{0}' AND '{1}' GROUP BY country "
           "UNION "
           "SELECT 'ROW' as country, SUM(impressions*average_position)/SUM(impressions) as average_position FROM "
           "t_google_search_average_position WHERE country not in ('US', 'UK', 'CA', 'DE', 'AU', 'FR', 'NL', "
           "'JP', 'IN', 'BR') AND activity_date between '{0}' AND '{1}' "
           "UNION "
           "SELECT 'WW' as country, SUM(impressions*average_position)/SUM(impressions) as average_position FROM "
           "t_google_search_average_position WHERE activity_date between '{0}' AND '{1}' ".format(start, end))
    average_position = mysqlconnect.fetch_data(qry, df=True)
    average_position['average_position'] /= 10.
    return average_position

In [6]:
row = lambda x: 'ROW' if x not in ('US', 'UK', 'GB', 'CA', 'DE', 'AU', 'FR', 'NL', 'JP', 'IN', 'BR') else x

In [7]:
def active_queries(start, end):
    qry = ("from semgoogle {0} {1} where impressions>0 group by account_country "
           "select distinct(keyword_id)".format(start, end))
    data = iql.dataframe(qry)
    data.columns = ['country', 'active_queries']
    data['country'] =  data['country'].apply(row)
    data = data.groupby(by=['country'])['active_queries'].sum()
    ww = pd.DataFrame([['WW', data.sum()]], columns=['country', 'active_queries'])
    ww = ww.groupby(by=['country'])['active_queries'].sum()
    return data.append(ww)

In [8]:
def traffic_data(start, end):
    qry_ga = ("SELECT country, SUM(sessions) as sessions FROM t_ga_unsampled_sessions_summarized "
              "WHERE country not in ('CMP', 'FORUM', 'GLOBAL', 'HIRE', 'REZ', 'ROW', 'ROWUS', 'SALARIES', 'SALARY') "
              "AND activity_date between '{0}' AND '{1}' AND channel='sem' "
              "GROUP BY country".format(start, end))
    data = mysqlconnect.fetch_data(qry_ga, df=True)
    data['country'] =  data['country'].apply(row)
    data = data.groupby(by=['country'])['sessions'].sum()
    ww = pd.DataFrame([['WW', data.sum()]], columns=['country', 'sessions'])
    ww = ww.groupby(by=['country'])['sessions'].sum()
    return data.append(ww)

In [9]:
def cost_revenue(start, end):
    qry = ("from semroi2 {0} {1} where medium='sem' "
           "sem_type='jobseeker' sid_source!='facebook' "
           "group by country select cost_millicents/100000, cost_clicks, "
           "(ga_revenue_est_millicents+sj_revenue_millicents)/100000".format(start, end))
    data = iql.dataframe(qry)
    data['country'] =  data['country'].apply(row)
    # cost
    cost =  data.groupby(by=['country'])['cost_millicents/100000'].sum()
    ww_cost = pd.DataFrame([['WW', cost.sum()]], columns=['country', 'cost'])
    ww_cost = ww_cost.groupby(by=['country'])['cost'].sum()
    cost = cost.append(ww_cost)
    # revenue
    revenue = data.groupby(by=['country'])['(ga_revenue_est_millicents+sj_revenue_millicents)/100000'].sum()
    ww_revenue = pd.DataFrame([['WW', revenue.sum()]], columns=['country', 'revenue'])
    ww_revenue = ww_revenue.groupby(by=['country'])['revenue'].sum()
    revenue = revenue.append(ww_revenue)
    # clicks
    clicks =  data.groupby(by=['country'])['cost_clicks'].sum()
    ww_clicks = pd.DataFrame([['WW', clicks.sum()]], columns=['country', 'clicks'])
    ww_clicks = ww_clicks.groupby(by=['country'])['clicks'].sum()
    clicks = clicks.append(ww_clicks)    
    return cost, revenue

In [10]:
avg_pos = get_average_position(start, end)

In [11]:
queries = active_queries(start, iql_end)

In [12]:
traffic = traffic_data(start, end)

In [13]:
cost, revenue = cost_revenue(start, iql_end)

In [14]:
def summary(country):
    print queries[country]
    print cost[country]
    print '{0:.2f}'.format(revenue[country])
    print traffic[country.replace('UK', 'GB')]
    print '{0:.1f}'.format(avg_pos[avg_pos.country==country]['average_position'].values[0])
    print '{0:.2f}'.format(revenue[country]/cost[country] - 1.)

In [26]:
summary('WW')

3740262
9127784.84942
27133780.18
78209792
1.8
1.97
